# Data Import

In [ ]:
def get_scraped_data(user_list, project_location):
    # get_scraped_data function begins

    # Process done by the function
    scraped_data = pd.DataFrame()

    for user in user_list:

        input_path = f"{project_location}Scraped Data\\{user}_data.csv"


        try:
            user_df = pd.read_csv(input_path, low_memory = False)
            print("Data from", user, "succesfully loaded with", len(user_df), "entries!")


        except FileNotFoundError:

            user_df = pd.DataFrame()

        scraped_data = pd.concat([scraped_data, user_df])

    initial_length = len(scraped_data)
    print("")
    print("Concatenated DF created with", initial_length, "entries!")
    
    return scraped_data

# Entry Cleaning

In [8]:
def count_non_standard_values(df, column_dict):
    # count_non_standard_values function begins
    
    # Process done by the function
    for curr_column in column_dict.keys():

        non_standard_values_count = df[curr_column].apply(lambda x: x not in column_dict[curr_column]).sum()
        
        print("Non Standard Values in " + str(curr_column) + ": " + str(non_standard_values_count))



In [4]:
def clean_from_rows_with_shifted_columns(df, allowance_dict = {"Umweltplakette" : ["1 (Keine)", "4 (Grün)", "3 (Gelb)", "2 (Rot)", np.nan],
    # clean_from_rows_with_shifted_columns function begins
                 "Antriebsart" : ['Heck', 'Front', 'Allrad', np.nan],
                 "Lackierung" : ['Metallic', 'Andere', 'Vollleder', 'Enthalten', 'Keine Angabe', np.nan],
                 "Türen" : ['2.0', '5.0', np.nan, '4.0', '3.0', '6.0', '1.0', '4', '5', '2222/BLN', '2222/BLT', '2', '3', '6', '9.0'],
                                                              "Scheckheftgepflegt" : ["Ja", "Nein", np.nan]}):
    
    # Process done by the function
    df1 = df.copy()
    previous_length = len(df1)

    for curr_feature in allowance_dict.keys():

        allowed_values = allowance_dict[curr_feature]
        df1 = df1[df1[curr_feature].isin(allowed_values)]

    new_length = len(df1)
    
    print("Dropped", previous_length - new_length, "entries due to shifted columns!")
    
    return df1
    

# Column Cleaning

Provide a function that shows the amount of missing data in every column.

In [5]:
def missing_values_info(df):
    # missing_values_info function begins
    
    # Process done by the function
    df1 = df.copy()
    # Count missing values in each column
    missing_counts = df1.isna().sum()

    # Calculate percentage of missing values in each column
    total_rows = len(df1)
    missing_percentages = np.round(missing_counts / total_rows, 3) * 100

    # Create a DataFrame to store missing values information
    missing_info_df = pd.DataFrame({
        'Missing Values': missing_counts,
        '% of Total Values': missing_percentages
    })
    missing_info_df = missing_info_df.sort_values(["Missing Values"], ascending = False)
    
    return missing_info_df

## Dropping Columns that contain "Unnamed"

In [2]:
def drop_unnamed_columns(df):
    # drop_unnamed_columns function begins
    
    # Process done by the function
    df1 = df.copy()
    # List comprehension to get column names containing "Unnamed" 
    unnamed_columns = [col for col in df.columns if 'Unnamed' in col]
    
    print("Dropping " + str(len(unnamed_columns)) + " columns:\n" + str(unnamed_columns))
    # Drop the unnamed columns
    df1 = df1.drop(columns=unnamed_columns, errors = 'ignore')
    
    return df1

## Dropping unwanted columns

In [7]:
def drop_unwanted_columns(df, columns_to_drop):
    # drop_unwanted_columns function begins
    
    # Process done by the function
    df1 = df.copy()
    df1 = df1.drop(columns = columns_to_drop, errors = "ignore")
    print("Dropped " + str(len(columns_to_drop)) + " columns:\n" + str(columns_to_drop))
          
    return df1

## Merge Columns

In [ ]:
def merge_columns(df, column1, column2):
    # merge_columns function begins
    
    # Process done by the function
    df1 = df.copy()
    df1[column1] = df1[column1].fillna(df1[column2])
    df1.drop(columns = [column2], inplace = True)
    
    return df1

# Converting Strings to Floats/Integers


## Convert Strings to other Strings

### Single Changes

In [ ]:
def replace_values_single(df, column_name, replacement_dict):
    # replace_values_single function begins
    
    # Process done by the function
    df1 = df.copy()
    
    for curr_value in replacement_dict.keys():
        
        
        df1[column_name] = df1[column_name].str.replace(curr_value, replacement_dict[curr_value], regex = False)
    
    return df1

### Multiple Changes

In [ ]:
def replace_values_multiple(df, replacement_dict):
    # replace_values_multiple function begins
    
    # Process done by the function
    df1 = df.copy()
    
    for column_name, replacements in replacement_dict.items():
        for curr_value, replacement in replacements.items():
            df1[column_name] = df1[column_name].str.replace(curr_value, replacement, regex = False)
    
    return df1

### Remove leading and trailing whitespaces

In [ ]:
def remove_trailing_whitespace(df, column_names):
    # remove_trailing_whitespace function begins
    
    # Process done by the function
    df1 = df.copy()
    
    for curr_column_name in column_names:
        
        df1[curr_column_name] = df1[curr_column_name].str.strip()
    
    return df1

### Replace Kraftstoff String with the most prominent Kraftstoff available in the initial Kraftstoff String

In [ ]:
def classify_fuel_type(fuel_type):
    # classify_fuel_type function begins
    if not isinstance(fuel_type, str):
        return 'Other'
    
    # Process done by the function
    fuel_type = fuel_type.lower()

    keywords = {
        'Diesel': ['diesel', 'biodiesel', 'pflanzenöl'],
        'Gas': ['autogas', 'lpg', 'flüssiggas', 'erdgas', 'cng', 'biogas'],
        'Super': ['super', 'super plus', 'benzin', 'e10', 'ethanol']
    }

    # Create a counter for all matches
    counter = Counter()

    for category, words in keywords.items():
        for word in words:
            if word in fuel_type:
                counter[category] += fuel_type.count(word)
    
    # If the counter is not empty, return the most common category
    if counter:
        return counter.most_common(1)[0][0]
    else:
        return 'Other'

    
# Apply the function to a DataFrame column
def transform_fuel_types(df, column_name):
    # transform_fuel_types function begins
    df1 = df.copy()
    df1[column_name] = df1[column_name].apply(classify_fuel_type)
    return df1

## Split Strings and turn to Floats

### Simple Strings

In [ ]:
def split_string_to_1integer(df, split_dict):
    # split_string_to_1integer function begins
    
    # Process done by the function
    df1 = df.copy()
    
    for curr_column in split_dict.keys():
        
        
        df1[curr_column] = df1[curr_column].str.split(split_dict[curr_column]).str[0].astype(float)

    
    return df1
        

In [ ]:
def split_string_to_1integer(df, split_dict):
    # split_string_to_1integer function begins
    
    # Process done by the function
    df1 = df.copy()
    
    def process_value_stromverbrauch(value):
    # process_value_stromverbrauch function begins
        
    # Process done by the function
        try:
            
            if np.isnan(value):
                
                return 0
                     
        except:

        
            try:
                
                result_value = value.split(split_dict[curr_column])[0]

                if result_value.count(".") > 1:

                    result_value = result_value.split(".")[0]

                result_value = float(result_value)
                
                return result_value

            except Exception as e:

                print(e)
                return 0
            
            
    for curr_column in split_dict.keys():
        
        if curr_column != "Stromverbrauch":
            df1[curr_column] = df1[curr_column].str.split(split_dict[curr_column]).str[0].astype(float)
        
        else:
            df1[curr_column] = df1[curr_column].apply(process_value_stromverbrauch)

    
    return df1

### Strings containing comma-seperated numbers

In [ ]:
def split_string_to_1integer_complex(df, split_dict):
    # split_string_to_1integer_complex function begins
    
    # Process done by the function
    df1 = df.copy()
    
    for curr_column in split_dict.keys():
        
        df1[curr_column] = df1[curr_column].str.split(split_dict[curr_column]).str[0]
        df1[curr_column] = df1[curr_column].str.split(".").str[0].astype(float)

    
    return df1

## Convert Miscellaneous to Floats

### Converting Kraftstoffverbrauch to Floats

In [ ]:
def convert_fuel_consumption(df, column_name):
    # convert_fuel_consumption function begins
    
    # Process done by the function
    df1 = df.copy()

    # Define regular expression patterns to extract komb., innerorts, and außerorts values
    komb_pattern = r'(\d+,\d+) l/100 km \(komb.\)'
    innerorts_pattern = r'(\d+,\d+) l/100 km \(innerorts\)'
    außerorts_pattern = r'(\d+,\d+) l/100 km \(außerorts\)'

    # Extract komb., innerorts, and außerorts values using regex
    komb_values = df1[column_name].str.extract(komb_pattern)
    innerorts_values = df1[column_name].str.extract(innerorts_pattern)
    außerorts_values = df1[column_name].str.extract(außerorts_pattern)

    # Replace commas with dots and convert to float
    komb_values = komb_values.replace(',', '.', regex=True).astype(float)
    innerorts_values = innerorts_values.replace(',', '.', regex=True).astype(float)
    außerorts_values = außerorts_values.replace(',', '.', regex=True).astype(float)

    # Calculate mean for rows containing both innerorts and außerorts values
    mean_values = (komb_values + innerorts_values + außerorts_values) / (komb_values.notna() + innerorts_values.notna() + außerorts_values.notna())

    # Replace NaNs in mean_values with komb_values
    mean_values.fillna(komb_values, inplace=True)

    # Replace NaNs in mean_values with innerorts_values
    mean_values.fillna(innerorts_values, inplace=True)

    # Replace NaNs in mean_values with außerorts_values
    mean_values.fillna(außerorts_values, inplace=True)

    # Replace NaNs in mean_values with 0
    mean_values.fillna(0, inplace=True)

    # Assign mean_values to the column in the DataFrame
    df1[column_name] = mean_values
    
    return df1


### Converting Barzahlungspreis to Floats

In [ ]:
def convert_barzahlungspreis_to_float(df, column_name):
    # convert_barzahlungspreis_to_float function begins
    
    # Process done by the function
    df1 = df.copy()
    
    # Replace missing values with NaN
    df1[column_name] = df1[column_name].fillna(float('nan'))
    
    # Remove unwanted characters from the specified column
    df1[column_name] = df1[column_name].str.replace('€', '', regex=True).str.replace('.', '', regex=True).str.replace(',', '.', regex=True).str.replace('-', '', regex=True).str.strip()
    
    # Convert to float, handling any potential errors
    try:
        df1[column_name] = df1[column_name].astype(float)
    except ValueError:
        # If conversion fails, try a different approach
        df1[column_name] = df1[column_name].apply(lambda x: ''.join(filter(str.isdigit, str(x))))
        df1[column_name] = df1[column_name].replace('', '0').astype(float)

    # Convert to integer if it's an integer value
    if df1[column_name].dtype == 'float64':
        df1[column_name] = df1[column_name].apply(lambda x: int(x) if x.is_integer() else x)
        
    df1[column_name] = df1[column_name].replace(0, np.nan, regex = False)
    
    return df1


# Missing Values

## Missing Values to Default Values

In [ ]:
def missing_values_to_default(df, replacement_dict):
    # missing_values_to_default function begins
    
    # Process done by the function
    df1 = df.copy()
    
    for curr_column in replacement_dict.keys():
        
        df1[curr_column] = df1[curr_column].fillna(replacement_dict[curr_column][0])
        
    return df1

## Missing values in specific columns to mean values 

### Mean Value for the Age of the car

In [9]:
def get_median_value_based_on_erstzulassung(df, column_to_mean, grouping_variable):
    # get_median_value_based_on_erstzulassung function begins
    
    # Process done by the function
    df1 = df.copy()
    
    mean_values = df1.groupby(df1[grouping_variable].astype(str).str[-4:])[column_to_mean].mean().round()
    year_means = dict(mean_values)
    
    df1[column_to_mean] = df1.apply(lambda row: year_means.get(str(row[grouping_variable])[-4:], row[column_to_mean]), axis=1)
    
    return df1

### Mean Values based on model and brand

In [ ]:
def fill_missing_with_mode_by_2groups(df, replacement_columns, group_column1, group_column2):
    # fill_missing_with_mode_by_2groups function begins
    
    # Process done by the function
    df1 = df.copy()
    
    for replacement_column in replacement_columns:

        # Calculate the mode of each group
        mode_per_group = df1.groupby([group_column1, group_column2])[replacement_column].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

        # Fill missing values using mode of corresponding group
        df1[replacement_column] = df1.apply(lambda row: mode_per_group.get((row[group_column1], row[group_column2]), np.nan) if pd.isnull(row[replacement_column]) else row[replacement_column], axis=1)

    return df1

### Mean Values based on model, brand and Leistung

In [ ]:
def fill_missing_with_mode_by_3groups(df, replacement_columns, group_column1, group_column2, group_column3):
    # fill_missing_with_mode_by_3groups function begins
    
    # Process done by the function
    df1 = df.copy()
    
    for replacement_column in replacement_columns:
        # Calculate the mode of each group
        mode_per_group = df1.groupby([group_column1, group_column2, group_column3])[replacement_column].apply(lambda x: x.mode().iloc[0] if not x.mode().empty else np.nan)

        # Fill missing values using mode of corresponding group
        df1[replacement_column] = df1.apply(lambda row: mode_per_group.get((row[group_column1], row[group_column2], row[group_column3]), np.nan) if pd.isnull(row[replacement_column]) else row[replacement_column], axis=1)

    return df1

# Split Multi Feature Columns

## And transform to count

In [ ]:
def count_strings_in_column(df, column_names):
    # count_strings_in_column function begins
    
    # Process done by the function
    # Define a regular expression pattern to identify the split points  
    pattern = r'(?<=[a-z])(?=[A-Z])'
    
    df1 = df.copy()
    
    for column_name in column_names:
        
        df2 = df1.copy()
        print("Current Feature:", column_name)

        # Split the string in the specified column based on the pattern
        split_values = df2[column_name].str.split(pattern, expand=True)

        # Rename the columns
        split_values.columns = [f"{column_name}_{i}" for i in range(1, split_values.shape[1] + 1)]

        # Concatenate the original DataFrame with the split values DataFrame
        df3 = pd.concat([df2, split_values], axis=1)


        counts = []
        
        for index, row in df3[[col for col in df3.columns if column_name in col]].iterrows():
            # Count the number of non-"None" values in the row
            count = sum(1 for value in row if value != "None" and not pd.isnull(value))
            counts.append(count)

        df1[column_name] = counts
    
    return df1

## And Create Extra Columns for them

In [1]:
def create_feature_columns(df, column_names):
    # create_feature_columns function begins
    
    # Process done by the function
    # Define a regular expression pattern to identify the split points  
    pattern = r'(?<=[a-z])(?=[A-Z])'
    
    df1 = df.copy()
    
    for column_name in column_names:
        
        df2 = df1.copy()
        print("Current Feature:", column_name)

        # Split the string in the specified column based on the pattern
        split_values = df2[column_name].dropna().str.split(pattern, expand=True)

        # Flatten the split values into a single list of features
        all_features = split_values.values.flatten()
        
        # Remove None and NaN values and get unique features
        all_features = [feature for feature in all_features if feature not in ["None", None] and pd.notna(feature)]
        unique_features = set(all_features)

        # Create a column for each unique feature
        for feature in unique_features:
            df1[f"{column_name}_{feature}"] = df2[column_name].apply(lambda x: 1 if isinstance(x, str) and feature in x else 0)
    
    print("Done!")
    return df1


In [2]:
def create_feature_columns(df, column_names):
    # create_feature_columns function begins
    # Define a regular expression pattern to identify the split points  
    pattern = r'(?<=[a-z])(?=[A-Z])'
    
    # Process done by the function
    df1 = df.copy()
    
    for column_name in column_names:
        
        df2 = df1.copy()
        print("Current Feature:", column_name)

        # Split the string in the specified column based on the pattern
        split_values = df2[column_name].dropna().str.split(pattern, expand=True)

        # Flatten the split values into a single list of features
        all_features = split_values.values.flatten()
        
        # Remove None and NaN values and get unique features
        all_features = [feature for feature in all_features if feature not in ["None", None] and pd.notna(feature)]
        unique_features = set(all_features)

        # Initialize a dictionary to hold the new columns
        feature_columns = {}

        # Create a column for each unique feature
        for feature in unique_features:
            feature_columns[f"{column_name}_{feature}"] = df2[column_name].apply(lambda x: 1 if isinstance(x, str) and feature in x else 0)
        
        # Concatenate the new feature columns to the original DataFrame
        df1 = pd.concat([df1, pd.DataFrame(feature_columns)], axis=1)
    
    print("Done!")
    return df1



# Date Columns

## Scraped "date" and "time" to timestamp

### Create New Variable

In [ ]:
def convert_to_timestamp(df, date_column, time_column):
    # convert_to_timestamp function begins
    
    # Process done by the function
    df1 = df.copy()
    
    df1["date_scraped"] = pd.to_datetime(df1[date_column] + ' ' + df1[time_column], format='%Y-%m-%d %H-%M-%S')
    
    return df1

## Erstzulassung to Age

### Split Erstzulassung to "Erstzulassung_Jahr" and "Erstzulassung_Monat"

In [11]:
def split_month_year(df, column_name):
    # split_month_year function begins
    
    # Process done by the function
    df1 = df.copy()
    
    # Split the date column into separate month and year columns
    df1[['Erstzulassung_Monat', 'Erstzulassung_Jahr']] = df1[column_name].str.split('/', expand=True)
    
    # Convert month and year columns to integers
    df1['Erstzulassung_Monat'] = df1['Erstzulassung_Monat'].astype(float).fillna(0).astype(int)
    df1['Erstzulassung_Jahr'] = df1['Erstzulassung_Jahr'].astype(float).fillna(0).astype(int)
    
    return df1

### Create Age Variable from that column (and delete Zulassungsmonat/Jahr)

In [ ]:
# transforming features to age:
def convert_zulassung_to_age(df, year_var, month_var):
    # convert_zulassung_to_age function begins
    
    # Process done by the function
    df1 = df.copy()
    
    age_list = []
    for i in range(0, len(df1[year_var])):
        
        scraping_time = df1["date_scraped"].iloc[i]
        # calculate difference in years since registration
        curr_years = scraping_time.year - df1[year_var].iloc[i]


        # calculate difference in months between this year and registration year
        curr_month_difference = scraping_time.month - df1[month_var].iloc[i]

        # add up variables to get age of the car
        curr_age = curr_years + curr_month_difference/12
        age_list.append(curr_age)
        
    df1["Alter"] = age_list
    df1["Alter"] = df1["Alter"].astype("float")
    df1["Alter"] = np.round(df1["Alter"], 2)
    
    return df1

# Drop Observations that contain NaNs 

# Outlier Handling

In [1]:
def remove_outliers(df, group_cols, target_col, lower_quantile=0.05, upper_quantile=0.95):
    # remove_outliers function begins
    
    # Process done by the function

    # work on copy
    df1 = df.copy()

    # Group by the specified columns
    grouped = df1.groupby(group_cols)
    
    # Create a mask to filter out the outliers
    mask = pd.Series([True] * len(df1), index=df1.index)
    
    # Iterate through each group
    for name, group in grouped:
        # Compute the lower and upper percentile values for the target column
        lower_threshold = group[target_col].quantile(lower_quantile)
        upper_threshold = group[target_col].quantile(upper_quantile)
        
        # Mark rows that are within the bounds
        mask.loc[group.index] = (group[target_col] >= lower_threshold) & (group[target_col] <= upper_threshold)
    
    # Return the DataFrame with outliers removed
    return df1[mask]

# Rename Columns

# Rearrange Columns

In [ ]:
def rearrange_columns(df, first_columns):
    # rearrange_columns function begins
    
    # Process done by the function
    df1 = df.copy()
    # Select the specified first columns
    first_columns_df = df1[first_columns]
    
    # Select the remaining columns
    remaining_columns = df1.drop(columns = first_columns)
    
    # Concatenate the specified first columns with the remaining columns
    rearranged_df = pd.concat([first_columns_df, remaining_columns], axis=1)
    
    return rearranged_df

# Data Cleaning Overview

In [ ]:
def data_cleaning_overview(initial_data, after_data):
    # data_cleaning_overview function begins
    
    # Process done by the function
    # calculate column metrics
    n_initial_columns = len(initial_data.columns)
    n_after_columns = len(after_data.columns)
    n_dropped_columns = n_initial_columns - n_after_columns
    perc_dropped_columns = np.round(n_dropped_columns/n_initial_columns, 2)*100
    
    # calculate observation metrics
    n_initial_obs = len(initial_data)
    n_after_obs = len(after_data)
    n_dropped_obs = n_initial_obs - n_after_obs
    perc_dropped_obs = np.round(n_dropped_obs/n_initial_obs, 2)*100
    
    
    print("################## Data Cleaning Overview ##################")
    print("")
    print(f"Initial Columns: {n_initial_columns}")
    print(f"Final Columns for model training: {n_after_columns}")
    print(f"Dropped {n_dropped_columns}({perc_dropped_columns}%) of all columns!")
    print("")
    print("")
    print(f"Initial Observations: {n_initial_obs}")
    print(f"Final Observations for model training: {n_after_obs}")
    print(f"Dropped {n_dropped_obs}({perc_dropped_obs}%) of all observations!")
    print("")
    print("################## /Data Cleaning Overview ##################")
          

# Data Saving

In [1]:
def save_clean_data(df, output_path):
    # save_clean_data function begins
    # Get the current datetime
    now = datetime.now()
    
    # Process done by the function
    # Format the datetime into the desired string format
    formatted_time = now.strftime("%Y_%m_%d %H-%M")
    output_file = f"{output_path}\cleaned_data_{formatted_time}.csv"
 
    df.to_csv(output_file, index = False)
    
    print(f"Succesfully saved cleaned dataframe to: \n\n{output_file} !")